In [1]:
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report


In [3]:
class CustomCSVDataset(Dataset):
    def __init__(self, root_dir):
        """
        Args:
            root_dir (string): Directory with all the data subdirectories.
        """
        self.data = []
        self.labels = []
        for label, folder_name in enumerate(['0', '1']):
            folder_path = os.path.join(root_dir, folder_name)
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.csv'):
                    file_path = os.path.join(folder_path, file_name)
                    csv_data = pd.read_csv(file_path)
                    csv_data = csv_data.drop('label', axis=1)
                    csv_data = csv_data.values
                    self.data.append(torch.tensor(csv_data, dtype=torch.float32))
                    self.labels.append(label)
    def __len__(self):
        return len(self.data)
 
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [4]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)  # Assuming binary classification (2 classes)
 
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:

class TransformerClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_heads=2, num_layers=1):
        super(TransformerClassifier, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc2 = nn.Linear(hidden_size, 2)  # Binary classification

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.unsqueeze(0)  # Add batch dimension for transformer input
        x = self.transformer_encoder(x)
        x = x.squeeze(0)  # Remove batch dimension
        x = self.fc2(x)
        return x

In [6]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.view(inputs.size(0), -1), labels  # Flatten inputs if necessary
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
 

In [7]:
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.view(inputs.size(0), -1), labels
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

In [8]:
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.view(inputs.size(0), -1), labels
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.numpy())
    # Generate and print the classification report
    print(classification_report(all_labels, all_predictions, target_names=['Class 0', 'Class 1']))

In [9]:
root_dir = 'dataset_postprocessing'  # Update with your data path
input_size = 64  # Replace with the actual number of features in your data
dataset = CustomCSVDataset(root_dir)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(dataset, batch_size=2, shuffle=False)

In [10]:
model_simple = SimpleClassifier(input_size=input_size)
criterion_simple = nn.CrossEntropyLoss()
optimizer_simple = optim.Adam(model_simple.parameters(), lr=0.001)

In [11]:
model_transformer = TransformerClassifier(input_size=input_size)
criterion_transformer = nn.CrossEntropyLoss()
optimizer_transformer = optim.Adam(model_transformer.parameters(), lr=0.001)

In [12]:
train_model(model_simple, train_loader, criterion_simple, optimizer_simple, num_epochs=100)

Epoch [1/100], Loss: 2.8672
Epoch [2/100], Loss: 0.4594
Epoch [3/100], Loss: 0.4384
Epoch [4/100], Loss: 10.3059
Epoch [5/100], Loss: 0.0649
Epoch [6/100], Loss: 0.5091
Epoch [7/100], Loss: 9.1010
Epoch [8/100], Loss: 8.1203
Epoch [9/100], Loss: 0.0252
Epoch [10/100], Loss: 0.0017
Epoch [11/100], Loss: 0.0275
Epoch [12/100], Loss: 0.0071
Epoch [13/100], Loss: 0.2696
Epoch [14/100], Loss: 0.0179
Epoch [15/100], Loss: 0.0000
Epoch [16/100], Loss: 0.0003
Epoch [17/100], Loss: 0.0570
Epoch [18/100], Loss: 0.1357
Epoch [19/100], Loss: 1.6407
Epoch [20/100], Loss: 0.0102
Epoch [21/100], Loss: 0.0556
Epoch [22/100], Loss: 0.0644
Epoch [23/100], Loss: 0.0000
Epoch [24/100], Loss: 0.0030
Epoch [25/100], Loss: 0.9634
Epoch [26/100], Loss: 0.0000
Epoch [27/100], Loss: 0.0581
Epoch [28/100], Loss: 0.0870
Epoch [29/100], Loss: 0.0006
Epoch [30/100], Loss: 0.0001
Epoch [31/100], Loss: 0.0095
Epoch [32/100], Loss: 0.0084
Epoch [33/100], Loss: 0.0470
Epoch [34/100], Loss: 0.0182
Epoch [35/100], Loss: 

In [17]:
train_model(model_transformer, train_loader, criterion_transformer, optimizer_transformer, num_epochs=100)

Epoch [1/100], Loss: 0.1809
Epoch [2/100], Loss: 0.2303
Epoch [3/100], Loss: 0.0033
Epoch [4/100], Loss: 0.0024
Epoch [5/100], Loss: 0.0060
Epoch [6/100], Loss: 0.0026
Epoch [7/100], Loss: 0.0050
Epoch [8/100], Loss: 0.1037
Epoch [9/100], Loss: 0.0021
Epoch [10/100], Loss: 1.9639
Epoch [11/100], Loss: 0.1213
Epoch [12/100], Loss: 0.0114
Epoch [13/100], Loss: 0.0242
Epoch [14/100], Loss: 0.7606
Epoch [15/100], Loss: 0.0439
Epoch [16/100], Loss: 0.0092
Epoch [17/100], Loss: 0.0022
Epoch [18/100], Loss: 0.2830
Epoch [19/100], Loss: 0.0016
Epoch [20/100], Loss: 0.0422
Epoch [21/100], Loss: 0.0024
Epoch [22/100], Loss: 0.0017
Epoch [23/100], Loss: 0.0043
Epoch [24/100], Loss: 0.0021
Epoch [25/100], Loss: 0.3733
Epoch [26/100], Loss: 0.0056
Epoch [27/100], Loss: 0.0019
Epoch [28/100], Loss: 0.0028
Epoch [29/100], Loss: 0.0009
Epoch [30/100], Loss: 0.0006
Epoch [31/100], Loss: 0.0007
Epoch [32/100], Loss: 0.0007
Epoch [33/100], Loss: 0.0008
Epoch [34/100], Loss: 0.0006
Epoch [35/100], Loss: 0

In [18]:
evaluate_model(model_transformer, train_loader)

              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00        37
     Class 1       1.00      1.00      1.00        15

    accuracy                           1.00        52
   macro avg       1.00      1.00      1.00        52
weighted avg       1.00      1.00      1.00        52



In [19]:
evaluate_model(model_simple, test_loader)

              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00        37
     Class 1       1.00      1.00      1.00        15

    accuracy                           1.00        52
   macro avg       1.00      1.00      1.00        52
weighted avg       1.00      1.00      1.00        52



In [23]:
model_save_path = "models_saved/model_simple.pth"
torch.save(model_simple.state_dict(), model_save_path)

In [22]:
model_save_path = "models_saved/model_transformer.pth"
torch.save(model_transformer.state_dict(), model_save_path)

In [24]:
def load_model(input_size, model_path, model_type ="simple"):
    if model_type == "simple":
        model = SimpleClassifier(input_size=input_size)
        model.load_state_dict(torch.load(model_path))
        model.eval()  # Set to evaluation mode
        print(f"Model loaded from {model_path}")
    elif model_type =='transformer':
        model = TransformerClassifier(input_size=input_size)
        model.load_state_dict(torch.load(model_path))
        model.eval()  # Set to evaluation mode
        print(f"Model loaded from {model_path}")
    else:
        raise TypeError('model_type must be "simple" or "transformer"')
    return model

In [25]:
loaded_model_simple = load_model(input_size=input_size, model_path="models_saved/model_simple.pth", model_type ="simple")

Model loaded from models_saved/model_simple.pth


In [26]:
loaded_model_transformer = load_model(input_size=input_size, model_path="models_saved/model_transformer.pth", model_type ="transformer")

Model loaded from models_saved/model_transformer.pth


In [27]:
def predict(model, input_data):
    with torch.no_grad():
        input_data = torch.tensor(input_data, dtype=torch.float32)
        input_data = input_data.view(1, -1)  # Reshape to match the input dimensions
        output = model(input_data)
        _, predicted_class = torch.max(output, 1)
        return predicted_class.item()

In [34]:
test_df = pd.read_csv(r"dataset_postprocessing\0\relax_0.csv")
test_df = test_df.drop('label', axis=1)
test_df = test_df.values


In [35]:
predicted_class = predict(loaded_model_simple, test_df)
print(f"Predicted class: {predicted_class}")

Predicted class: 0


In [36]:
predicted_class = predict(loaded_model_transformer, test_df)
print(f"Predicted class: {predicted_class}")

Predicted class: 0
